In [ ]:
# --- Librerías básicas ---
import pandas as pd
import numpy as np

# --- Librerías de Deep Learning ---
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# --- Librerías de Preprocesamiento ---
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# --- Librerías de Visualización ---
import matplotlib.pyplot as plt
import seaborn as sns

# Confirmar que se han importado correctamente
print("✅ Librerías importadas correctamente.")



✅ Librerías importadas correctamente.


In [ ]:
# --- Cargar dataset desde Google Drive ---
df = pd.read_csv("https://drive.google.com/uc?export=download&id=18glOMWxHUOPDQa6t2mwRG6vJN1GbpiIA")

# --- Configuración para visualizar mejor los DataFrames ---
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# --- Verificar las primeras filas del DataFrame ---
print("🔎 Primeras filas del DataFrame original:")
print(df.head())



🔎 Primeras filas del DataFrame original:
                                      url              title   type  \
0  https://www.netflix.com/title/60000724       Forrest Gump  movie   
1   https://www.netflix.com/title/1154386  The Fifth Element  movie   
2  https://www.netflix.com/title/60031236  Kill Bill: Vol. 1  movie   
3  https://www.netflix.com/title/70021659            Jarhead  movie   
4   https://www.netflix.com/title/1080395         Unforgiven  movie   

                      genres  releaseYear     imdbId  imdbAverageRating  \
0             Drama, Romance       1994.0  tt0109830                8.8   
1  Action, Adventure, Sci-Fi       1997.0  tt0119116                7.6   
2    Action, Crime, Thriller       2003.0  tt0266697                8.2   
3      Biography, Drama, War       2005.0  tt0418763                7.0   
4             Drama, Western       1992.0  tt0105695                8.2   

   imdbNumVotes                                 availableCountries  
0     231322

In [ ]:
# --- Eliminar filas con valores nulos ---
df = df.dropna().reset_index(drop=True)

# --- Ajustar tipos de columnas ---
df['releaseYear'] = df['releaseYear'].astype('Int64')
df['imdbNumVotes'] = df['imdbNumVotes'].astype('Int64')

# --- Eliminar columnas innecesarias ---
df = df.drop(['url', 'imdbId', 'availableCountries'], axis=1)

# --- Mostrar información general después del preprocesamiento ---
print("✅ Preprocesamiento básico realizado.")
print(df.info())



✅ Preprocesamiento básico realizado.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6697 entries, 0 to 6696
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              6697 non-null   object 
 1   type               6697 non-null   object 
 2   genres             6697 non-null   object 
 3   releaseYear        6697 non-null   Int64  
 4   imdbAverageRating  6697 non-null   float64
 5   imdbNumVotes       6697 non-null   Int64  
dtypes: Int64(2), float64(1), object(3)
memory usage: 327.1+ KB
None


In [ ]:
# --- Crear nueva columna para distinguir entre películas y series ---
df['type_new'] = np.where(df['type'] == 'movie', 0, 1)

# --- Función para codificar géneros relevantes ---
def encode_new(genres):
    encoding = [0, 0, 0]  # Drama, Comedy, Other
    if pd.isna(genres):
        return [None, None, None]
    elif 'Drama' in genres and 'Comedy' not in genres:
        encoding[0] = 1
    elif 'Comedy' in genres and 'Drama' not in genres:
        encoding[1] = 1
    elif 'Comedy' not in genres and 'Drama' not in genres:
        encoding[2] = 1
    else:
        return [None, None, None]
    return encoding

# --- Aplicar codificación de géneros ---
df[['drama', 'comedy', 'other']] = pd.DataFrame(df['genres'].apply(encode_new).tolist(), index=df.index)

# --- Eliminar filas donde la codificación de géneros no fue posible ---
df = df.dropna(subset=['drama', 'comedy', 'other'])

# --- Eliminar columnas originales ---
df = df.drop(['genres', 'type'], axis=1)

# --- Mostrar primeras filas para comprobar ---
print("✅ Codificación de géneros completada.")
print(df[['drama', 'comedy', 'other']].head())



✅ Codificación de géneros completada.
   drama  comedy  other
0    1.0     0.0    0.0
1    0.0     0.0    1.0
2    0.0     0.0    1.0
3    1.0     0.0    0.0
4    1.0     0.0    0.0


In [ ]:
# --- Función para eliminar valores atípicos basados en 3-sigma ---
def remove_outliers_3sigma(df, columns):
    for col in columns:
        mean = df[col].mean()
        std = df[col].std()
        lower_bound = mean - 3 * std
        upper_bound = mean + 3 * std
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

# --- Aplicar eliminación de outliers ---
df = remove_outliers_3sigma(df, ['releaseYear', 'imdbNumVotes'])

# --- Mostrar tamaño del DataFrame después de eliminar outliers ---
print(f"✅ Outliers eliminados. Tamaño del DataFrame: {df.shape}")



✅ Outliers eliminados. Tamaño del DataFrame: (5548, 8)


In [ ]:
# --- Función para categorizar el rating de IMDB ---
def categorize_imdb_rating(rating):
    if rating < 6:
        return "Mal"
    elif 6 <= rating < 7:
        return "Bien"
    elif 7 <= rating < 8:
        return "Muy bien"
    elif 8 <= rating <= 10:
        return "Excelente"
    else:
        return "No categorizado"

# --- Aplicar función ---
df['rating_category'] = df['imdbAverageRating'].apply(categorize_imdb_rating)

# --- Ver distribución de clases ---
print("✅ Categorías de rating creadas:")
print(df['rating_category'].value_counts())



✅ Categorías de rating creadas:
rating_category
Bien         1984
Muy bien     1787
Mal          1314
Excelente     463
Name: count, dtype: int64


In [ ]:
# --- Estandarizar releaseYear e imdbNumVotes ---
scaler = StandardScaler()
columns_to_standardize = ['releaseYear', 'imdbNumVotes']
df[columns_to_standardize] = scaler.fit_transform(df[columns_to_standardize])

# --- Mostrar estadísticos después de estandarización ---
print("✅ Variables numéricas estandarizadas:")
print(df[columns_to_standardize])



✅ Variables numéricas estandarizadas:
      releaseYear  imdbNumVotes
1       -1.658254      4.252866
3       -0.809996      1.418538
4       -2.188416      3.572972
6       -1.340158      1.837221
7       -0.809996      1.858180
...           ...           ...
6691     1.204617     -0.540043
6692     1.204617     -0.537759
6693     1.204617     -0.537090
6694     1.204617     -0.520272
6696     1.204617     -0.536654

[5548 rows x 2 columns]


In [ ]:
# --- Seleccionar 5.000 filas de manera aleatoria ---
df = df.sample(5000, random_state=40).reset_index(drop=True)

# --- Definir X (features) e y (target) ---
X = df[['type_new', 'releaseYear', 'imdbNumVotes', 'drama', 'comedy', 'other']]
y = df['rating_category']

# --- Codificar etiquetas de y ---
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# --- Dividir en conjuntos de entrenamiento y test ---
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=40)

# --- Convertir datos a tensores ---
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# --- Crear dataset y dataloader ---
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# --- Mostrar tamaños de los datasets ---
print(f"✅ Datos preparados. Tamaño X_train: {X_train_tensor.shape}, Tamaño y_train: {y_train_tensor.shape}")



✅ Datos preparados. Tamaño X_train: torch.Size([4000, 6]), Tamaño y_train: torch.Size([4000])


In [ ]:
# --- Definir Red Neuronal ---
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(6, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 4)  # 4 clases posibles

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

best_accuracy = 0
best_params = {}

for lr in [0.001, 0.01, 0.1]:
    for momentum in [0.5, 0.9]:
        model = NeuralNetwork()
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

        # Entrenamiento (solo 5 epochs para evaluar rápido)
        for epoch in range(5):
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

        # Evaluación en test
        with torch.no_grad():
            outputs = model(X_test_tensor)
            _, predicted = torch.max(outputs, 1)
            accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)

        print(f"lr={lr}, momentum={momentum}, accuracy={accuracy:.4f}")

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'lr': lr, 'momentum': momentum}

print(f"\n🚀 Mejor combinación encontrada: {best_params}, con accuracy: {best_accuracy:.4f}")




lr=0.001, momentum=0.5, accuracy=0.3190
lr=0.001, momentum=0.9, accuracy=0.3880
lr=0.01, momentum=0.5, accuracy=0.3810
lr=0.01, momentum=0.9, accuracy=0.4470
lr=0.1, momentum=0.5, accuracy=0.4520
lr=0.1, momentum=0.9, accuracy=0.4410

🚀 Mejor combinación encontrada: {'lr': 0.1, 'momentum': 0.5}, con accuracy: 0.4520


In [ ]:
# --- Entrenar la red neuronal ---
num_epochs = 30

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Mostrar pérdida cada 5 épocas
    if (epoch + 1) % 5 == 0:
        print(f"📈 Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("✅ Entrenamiento finalizado.")



📈 Epoch [5/30], Loss: 1.1093
📈 Epoch [10/30], Loss: 1.0995
📈 Epoch [15/30], Loss: 1.0994
📈 Epoch [20/30], Loss: 1.0924
📈 Epoch [25/30], Loss: 1.0850
📈 Epoch [30/30], Loss: 1.0839
✅ Entrenamiento finalizado.


In [ ]:
# --- Invertir codificación de etiquetas para mostrar predicciones ---
predicted_labels = label_encoder.inverse_transform(predicted.numpy())
real_labels = label_encoder.inverse_transform(y_test_tensor.numpy())

# --- Crear DataFrame de comparación ---
results = pd.DataFrame({'Real': real_labels, 'Predicho': predicted_labels})

# --- Mostrar 15 ejemplos ---
print("📋 Muestra de predicciones reales vs predichas:")
print(results.sample(15, random_state=40))


📋 Muestra de predicciones reales vs predichas:
          Real  Predicho
204   Muy bien      Bien
71        Bien  Muy bien
594       Bien      Bien
672   Muy bien  Muy bien
14    Muy bien  Muy bien
64        Bien      Bien
340   Muy bien      Bien
135       Bien       Mal
350        Mal  Muy bien
976       Bien  Muy bien
181       Bien  Muy bien
538       Bien      Bien
246       Bien      Bien
982  Excelente  Muy bien
366        Mal      Bien


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test_tensor, predicted))
print(classification_report(y_test_tensor, predicted, target_names=label_encoder.classes_))


[[127   0  44 192]
 [  7   8   1  72]
 [105   0  64  62]
 [ 61   1  14 242]]
              precision    recall  f1-score   support

        Bien       0.42      0.35      0.38       363
   Excelente       0.89      0.09      0.16        88
         Mal       0.52      0.28      0.36       231
    Muy bien       0.43      0.76      0.55       318

    accuracy                           0.44      1000
   macro avg       0.56      0.37      0.36      1000
weighted avg       0.49      0.44      0.41      1000



In [ ]:
# --- PREDICCIÓN SOBRE PELÍCULAS FICTICIAS ---

# Datos originales de ejemplo (sin estandarizar aún)
nuevas_pelis = pd.DataFrame({
    'type_new': [0, 1],                 # 0: película, 1: serie
    'releaseYear': [2015, 2020],        # años de estreno
    'imdbNumVotes': [50000, 5000],      # número de votos
    'drama': [1, 0],                    # género drama
    'comedy': [0, 1],                   # género comedia
    'other': [0, 0]                     # otros géneros
})

# Estandarizar releaseYear y imdbNumVotes igual que el conjunto original
nuevas_pelis[['releaseYear', 'imdbNumVotes']] = scaler.transform(nuevas_pelis[['releaseYear', 'imdbNumVotes']])

# Convertir a tensor
nuevas_tensor = torch.tensor(nuevas_pelis.values, dtype=torch.float32)

# Predecir
with torch.no_grad():
    predicciones = model(nuevas_tensor)
    _, clases_predichas = torch.max(predicciones, 1)

# Decodificar etiquetas
categorias_predichas = label_encoder.inverse_transform(clases_predichas.numpy())

# Mostrar resultados
for i, categoria in enumerate(categorias_predichas, 1):
    print(f"Película {i} → Predicción de calidad IMDb: {categoria}")


Película 1 → Predicción de calidad IMDb: Bien
Película 2 → Predicción de calidad IMDb: Bien
